# Quickstart Example

This is the same tutorial that you can find [here](https://genworlds.com/docs/get-started/quickstart) but with a `ChatInterfaceWorld` so it can be connected to the `vue-gui` frontend provided in the [GenWorlds-Community](https://github.com/yeagerai/genworlds-community/) repo.

So I'll omit most of the text cells. If you want to read the tutorial, go to [this page](https://genworlds.com/docs/get-started/quickstart).

In [1]:
from datetime import datetime
import threading
from typing import List
from time import sleep
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

In [2]:
from genworlds.worlds.concrete.community_chat_interface.world import ChatInterfaceWorld

# Define the World
hello_world = ChatInterfaceWorld(
    name="Hello World",
    description="A world for testing the library.",
)

# Launch the World
hello_world.launch()

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:     Started server process [13060]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:7456 (Press CTRL+C to quit)
INFO:     ('127.0.0.1', 50999) - "WebSocket /ws" [accepted]
INFO:     connection open
INFO:websocket:Websocket connected
[1974b812-facb-4e5b-b621-aade09bbc3b3 Thread] Connected to world socket server ws://127.0.0.1:7456/ws
INFO:1974b812-facb-4e5b-b621-aade09bbc3b3 Thread:Connected to world socket server ws://127.0.0.1:7456/ws


In [3]:
from genworlds.agents.concrete.basic_assistant.utils import generate_basic_assistant
from genworlds.worlds.concrete.base.actions import UserSpeaksWithAgentEvent

description = """Agent that helps the user to answer questions."""

# Generate a basic assistant (you have to provide the openai_api_key)
agent1 = generate_basic_assistant(
    agent_name="agent1",
    description=description,
    openai_api_key=openai_api_key
)

agent1.state_manager.state.is_asleep = True

# Add wakeup events to the agents so you can interact with the agent after is sleeping
agent1.add_wakeup_event(event_class=UserSpeaksWithAgentEvent)

## Attach the agent to the World
hello_world.add_agent(agent1)

INFO:     ('127.0.0.1', 51000) - "WebSocket /ws" [accepted]
INFO:     connection open
INFO:websocket:Websocket connected
[agent1 Thread] Connected to world socket server ws://127.0.0.1:7456/ws
INFO:agent1 Thread:Connected to world socket server ws://127.0.0.1:7456/ws


In [4]:
from genworlds.events.abstracts.event import AbstractEvent
from genworlds.events.abstracts.action import AbstractAction
from genworlds.objects.abstracts.object import AbstractObject

# Create two events and one action
# Event that the agent will use to ask for the sum of two numbers
class AgentWantsToAddTwoNumbers(AbstractEvent):
    event_type = "agent_wants_to_add_two_numbers_event"
    description = "Sends two float numbers to be added in a deterministic way."
    number1 : float
    number2 : float

# Event that the SimpleCalculator will send to the agent with the sum of the two numbers
class SendAddedNumbersToAgent(AbstractEvent):
    event_type = "send_added_numbers_to_agent_event"
    description = "Sends the requested deterministic sum of the two numbers to the agent"
    sum : float

# The action that will be triggered by the event and will perform the addition of the two numbers
class AddTwoNumbers(AbstractAction):
    trigger_event_class = AgentWantsToAddTwoNumbers
    description = "Is used to add two float numbers in a deterministic way."
    
    def __init__(self, host_object: AbstractObject):
        self.host_object = host_object
    
    def __call__(self, event:AgentWantsToAddTwoNumbers):
        event = SendAddedNumbersToAgent(
            sender_id=self.host_object.id,
            target_id=event.sender_id,
            sum = event.number1 + event.number2,
        )
        self.host_object.send_event(event)

In [5]:
# Define the SimpleCalculator Class
class SimpleCalculator(AbstractObject):
    def __init__(self, id:str):
        actions = [AddTwoNumbers(host_object=self)]
        super().__init__(name="Simple Calculator", 
                         id=id, 
                         description="Object used to do some calculations such as adding two numbers.",
                         actions=actions
                         )

# Instantiate the SimpleCalculator Class
simple_calculator = SimpleCalculator(id="simple_calculator")

# Incorporate the simple_calculator object into the World
hello_world.add_object(simple_calculator)

INFO:     ('127.0.0.1', 51001) - "WebSocket /ws" [accepted]
INFO:     connection open
INFO:websocket:Websocket connected
[simple_calculator Thread] Connected to world socket server ws://127.0.0.1:7456/ws
INFO:simple_calculator Thread:Connected to world socket server ws://127.0.0.1:7456/ws
INFO:     ('127.0.0.1', 51011) - "WebSocket /ws" [accepted]
INFO:     connection open


{"event_type":"user_requests_screens_to_world","description":"The user requests the screens to the world.","sender_id":"test_user","target_id":null,"created_at":"2023-10-13T08:50:38.225Z"}
{"event_type":"agent_wants_updated_state","description":"Agent wants to update its state.","sender_id":"test_user","target_id":null,"created_at":"2023-10-13T08:50:38.225Z"}
{"event_type": "world_sends_screens_to_user", "description": "The world sends the screens to the user.", "summary": null, "created_at": "2023-10-13T09:50:38.228058", "sender_id": "1974b812-facb-4e5b-b621-aade09bbc3b3", "target_id": null, "screens_config": {"name": "RAG World", "screens": [{"name": "Chat Room", "has_input": false, "screen_type": "event_history", "tracked_events": [{"event_type": "user_speaks_with_agent_event"}, {"event_type": "agent_goes_to_sleep"}, {"event_type": "agent_speaks_with_agent_event"}, {"event_type": "agent_speaks_with_user_event"}]}, {"name": "All events", "has_input": false, "screen_type": "event_hist



> Entering new LLMChain chain...
Prompt after formatting:
System: You are agent1, Agent that helps the user to answer questions..

System: You are embedded in a simulated world with those properties 

System: Those are your goals: 
['Starts waiting and sleeps till the user starts a new question.', "Once agent1 receives a user's question, he makes sure to have all the information before sending the answer to the user.", 'When agent1 has all the required information, he speaks to the user with the results through the agent_speaks_with_user_event.', 'After sending the response, he waits for the next user question.', 'If you have been waiting for any object or entity to send you an event for over 30 seconds, you will wait and sleep until you receive a new event.']

System: And this is your current plan to achieve the goals: 
[]

System: Here is your memories of all the events that you remember from being in this simulation: 


# Your Memories

## Last events from oldest to most recent

{



> Entering new LLMChain chain...
Prompt after formatting:
System: You are agent1, Agent that helps the user to answer questions..

System: You are embedded in a simulated world with those properties 

System: Those are your goals: 
['Starts waiting and sleeps till the user starts a new question.', "Once agent1 receives a user's question, he makes sure to have all the information before sending the answer to the user.", 'When agent1 has all the required information, he speaks to the user with the results through the agent_speaks_with_user_event.', 'After sending the response, he waits for the next user question.', 'If you have been waiting for any object or entity to send you an event for over 30 seconds, you will wait and sleep until you receive a new event.']

System: And this is your current plan to achieve the goals: 
['simple_calculator:AddTwoNumbers', 'agent1:AgentSpeaksWithUser', 'agent1:AgentGoesToSleep']

System: Here is your memories of all the events that you remember from 



> Entering new LLMChain chain...
Prompt after formatting:
System: You are agent1, Agent that helps the user to answer questions..

System: You are embedded in a simulated world with those properties 

System: Those are your goals: 
['Starts waiting and sleeps till the user starts a new question.', "Once agent1 receives a user's question, he makes sure to have all the information before sending the answer to the user.", 'When agent1 has all the required information, he speaks to the user with the results through the agent_speaks_with_user_event.', 'After sending the response, he waits for the next user question.', 'If you have been waiting for any object or entity to send you an event for over 30 seconds, you will wait and sleep until you receive a new event.']

System: And this is your current plan to achieve the goals: 
['simple_calculator:AddTwoNumbers', 'agent1:AgentSpeaksWithUser', 'agent1:AgentGoesToSleep']

System: Here is your memories of all the events that you remember from 


> Finished chain.
{"event_type": "agent_speaks_with_user_event", "description": "An agent speaks with the user.", "summary": null, "created_at": "2023-10-13T09:52:08.997802", "sender_id": "agent1", "target_id": null, "message": "The result of adding 2.45 and 7.59 is 10.04"}
{"event_type": "agent_wants_updated_state", "description": "Agent wants to update its state.", "summary": null, "created_at": "2023-10-13T09:52:08.998774", "sender_id": "agent1", "target_id": "1974b812-facb-4e5b-b621-aade09bbc3b3"}
{"event_type": "world_sends_available_entities_event", "description": "Send available entities.", "summary": null, "created_at": "2023-10-13T09:52:09.000775", "sender_id": "1974b812-facb-4e5b-b621-aade09bbc3b3", "target_id": "agent1", "available_entities": {"1974b812-facb-4e5b-b621-aade09bbc3b3": {"id": "1974b812-facb-4e5b-b621-aade09bbc3b3", "entity_type": "WORLD", "entity_class": "ChatInterfaceWorld", "name": "Hello World", "description": "A world for testing the library."}, "agent1": 


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are agent1, Agent that helps the user to answer questions..
System: You are embedded in a simulated world with those properties 
System: Those are your goals: 
['Starts waiting and sleeps till the user starts a new question.', "Once agent1 receives a user's question, he makes sure to have all the information before sending the answer to the user.", 'When agent1 has all the required information, he speaks to the user with the results through the agent_speaks_with_user_event.', 'After sending the response, he waits for the next user question.', 'If you have been waiting for any object or entity to send you an event for over 30 seconds, you will wait and sleep until you receive a new event.']
System: And this is your current plan to achieve the goals: 
['agent1:AgentGoesToSleep']
System: Here is your memories of all the events that you remember from being in this simulation: 


# Your Memories

## F



> Entering new LLMChain chain...
Prompt after formatting:
System: You are agent1, Agent that helps the user to answer questions..

System: You are embedded in a simulated world with those properties 

System: Those are your goals: 
['Starts waiting and sleeps till the user starts a new question.', "Once agent1 receives a user's question, he makes sure to have all the information before sending the answer to the user.", 'When agent1 has all the required information, he speaks to the user with the results through the agent_speaks_with_user_event.', 'After sending the response, he waits for the next user question.', 'If you have been waiting for any object or entity to send you an event for over 30 seconds, you will wait and sleep until you receive a new event.']

System: And this is your current plan to achieve the goals: 
['agent1:AgentGoesToSleep']

System: Here is your memories of all the events that you remember from being in this simulation: 


# Your Memories

## Full Summary



